<a href="https://colab.research.google.com/github/ramanp83/Processing-Transcript_file/blob/main/09_BLSTM_52.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install contractions

In [ ]:
from IPython import get_ipython
from IPython.display import display
import pandas as pd
import os
import re
import nltk
import tensorflow as tf
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import contractions  # Import contractions
import tensorflow_hub as hub
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Bidirectional, Input
from tensorflow.keras.layers import Masking

# Download necessary NLTK resources
# !pip install nltk==3.8.1
# nltk.download('punkt')
# nltk.download('stopwords')
# nltk.download('punkt_tab') # Download punkt_tab

# Define stopwords and contraction mappings
stop_words = set(stopwords.words('english'))

In [ ]:
'''format for K's directory'''
# def load_transcripts(data_dir):
#     transcripts = []
#     target_folders = [f"{i}_P" for i in range(300, 305)]  # Folders to process
#     target_file = "300_TRANSCRIPT.csv"  # File to read within each folder

#     for folder_name in target_folders:  # e.g. 300_P,etc
#         folder_path = os.path.join(data_dir, folder_name)  # concatenate
#         # e.g. data_dir + folder_name = full path i.e. till 300_P
#         # Check if the target folder exists
#         if os.path.isdir(folder_path):  # if folder_path exists
#             file_path = os.path.join(folder_path, target_file)

#             # Check if the target file exists within the folder
#             if os.path.isfile(file_path):  # give full path to 300_Transcript.csv
#                 df = pd.read_csv(file_path, sep='\t', names=['start', 'end', 'speaker', 'value'])
#                 # Add Participant_ID column
#                 participant_id = int(folder_name.split('_')[0])
#                 df['Participant_ID'] = participant_id
#                 transcripts.append(df)
#             else:
#                 print(f"File '{target_file}' not found in folder '{folder_name}'")
#         else:
#             print(f"Folder '{folder_name}' not found in '{data_dir}'")

#     return transcripts


# transcripts = load_transcripts("/content/drive/MyDrive/Transcipts")

"format for K's directory"

In [ ]:
import os
import pandas as pd

def load_transcripts(data_dir):
    transcripts = []
    # Target files (300 to 352, excluding 342 and 344)
    target_files = [f"{i}_TRANSCRIPT.csv" for i in range(300, 311) if i not in [342, 344]]

    for file_name in target_files:
        file_path = os.path.join(data_dir, file_name)

        if os.path.isfile(file_path):
            df = pd.read_csv(file_path, sep='\t', names=['start', 'end', 'speaker', 'value'])
            # Add Participant_ID column
            participant_id = int(file_name.split('_')[0])
            df['Participant_ID'] = participant_id
            transcripts.append(df)
        else:
            print(f"File '{file_name}' not found in folder '{data_dir}'")

    return transcripts

# Call the function with the updated directory
transcripts = load_transcripts("/content/drive/MyDrive/datasset/transcript")

In [ ]:
contractions_dict = {
    "can't": "cannot", "won't": "will not", "n't": " not", "i'm": "i am",
    "you're": "you are", "it's": "it is", "that's": "that is", "let's": "let us",
    "aren't": "are not", "isn't": "is not", "wasn't": "was not", "weren't": "were not",
    "haven't": "have not", "hasn't": "has not", "hadn't": "had not", "don't": "do not",
    "doesn't": "does not", "didn't": "did not", "won't": "will not", "wouldn't": "would not",
    "shouldn't": "should not", "couldn't": "could not", "mightn't": "might not",
    "mustn't": "must not", "shan't": "shall not", "she's": "she is", "he's": "he is",
    "we're": "we are", "they're": "they are", "i'll": "i will", "you'll": "you will",
    "he'll": "he will", "she'll": "she will", "we'll": "we will", "they'll": "they will",
    "i'd": "i would", "you'd": "you would", "he'd": "he would", "she'd": "she would",
    "we'd": "we would", "they'd": "they would", "there's": "there is", "here's": "here is",
    "what's": "what is", "who's": "who is", "how's": "how is", "where's": "where is",
    "why's": "why is", "let's": "let us", "y'all": "you all", "d'you": "do you",
    "gonna": "going to", "gotta": "got to", "wanna": "want to", "c'mon": "come on",
    "ain't": "is not", "o'clock": "of the clock", "'cause": "because",
    "could've": "could have", "should've": "should have", "would've": "would have",
    "might've": "might have", "must've": "must have", "you've": "you have",
    "we've": "we have", "they've": "they have", "i've": "i have",
    "it'd": "it would", "there'd": "there would", "here'd": "here would",
    "that'd": "that would", "this'd": "this would", "who'd": "who would",
    "what'd": "what did", "where'd": "where did", "how'd": "how did"
}

In [ ]:
def expand_contractions_manual(text):
    for contraction, full_form in contractions_dict.items():
        text = text.replace(contraction, full_form)
    return text


def remove_stopwords(text):
    words = word_tokenize(text)
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)


def clean_text_enhanced(text):
    # Convert to string if not already, handling NaN and other types
    text = str(text)
    text = re.sub(r"<.*?>", "", text)  # Remove incomplete word annotations
    text = text.replace("XXX", "[UNK]")  # Replace unrecognizable words
    text = expand_contractions_manual(text)  # Expand contractions manually
    text = remove_stopwords(text)  # Remove stopwords
    return text


# Apply enhanced cleaning to all transcripts
for df in transcripts:
    if 'value' in df.columns:
        df['processed_text'] = df['value'].apply(clean_text_enhanced)
    else:
        print("Column 'value' not found in DataFrame.")

print("Transcript 0th index is are :", transcripts[0].head(1))


def expand_contractions(text):
    return contractions.fix(text)


# Load Universal Sentence Encoder
embed_model = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

def generate_embeddings(text_list):
    embeddings = embed_model(text_list)
    print("Shape of embeddings:", embeddings.shape)  # Check shape
    return np.array(embeddings)

MAX_TIMESTEPS = 400
EMBEDDING_DIM = 512

# Generate embeddings for processed text
for df in transcripts:
    if 'processed_text' in df.columns:
        all_embeddings = generate_embeddings(df['processed_text'].tolist())

        # Reshape to (num_samples, embedding_dim) if necessary
        all_embeddings = all_embeddings.reshape(-1, EMBEDDING_DIM)

        df['embeddings'] = all_embeddings.tolist()# Make sure embeddings are stored as lists of 512-dimensional vectors
         # Convert to list of lists
    else:
        print("Column 'processed_text' not found in DataFrame.")



Transcript 0th index is are :         start        end  speaker  value  Participant_ID processed_text
0  start_time  stop_time  speaker  value             300          value
Shape of embeddings: (175, 512)
Shape of embeddings: (182, 512)
Shape of embeddings: (187, 512)
Shape of embeddings: (192, 512)
Shape of embeddings: (205, 512)
Shape of embeddings: (406, 512)
Shape of embeddings: (214, 512)
Shape of embeddings: (278, 512)
Shape of embeddings: (225, 512)
Shape of embeddings: (177, 512)
Shape of embeddings: (241, 512)


In [ ]:
def pad_sequences_uniform(embeddings):
       # Reshape if needed (avoid unnecessary copy if already 3D)
       if embeddings.ndim == 2:
           embeddings = embeddings.reshape(embeddings.shape[0], 1, EMBEDDING_DIM)

       # Pad sequences (use in-place modification if possible)
       padded_embeddings = pad_sequences(embeddings, maxlen=MAX_TIMESTEPS,
                                        dtype='float32', padding='post',
                                        truncating='post')
       return padded_embeddings


# Merge metadata with transcript data
train_metadata = pd.read_csv('/content/drive/MyDrive/Transcipts/train_split_Depression_AVEC2017.csv')
dev_metadata = pd.read_csv('/content/drive/MyDrive/Transcipts/dev_split_Depression_AVEC2017.csv')
test_metadata = pd.read_csv('/content/drive/MyDrive/Transcipts/test_split_Depression_AVEC2017.csv')

# Concatenate all transcripts
all_transcripts = pd.concat(transcripts)

In [ ]:
# Merge metadata with transcript data based on Participant_ID
merged_data = pd.merge(train_metadata, all_transcripts, on='Participant_ID', how='inner')
participants_with_transcripts = all_transcripts['Participant_ID'].unique()
merged_data = merged_data[merged_data['Participant_ID'].isin(participants_with_transcripts)]


In [ ]:
# # Iterate through each participant group
# for participant_id, group_data in participant_groups:
#     # Get all embeddings for the participant
#     participant_embeddings = group_data['embeddings'].tolist()

#     # Pad the embeddings for the participant
#     padded_participant_embeddings = pad_sequences_uniform(participant_embeddings)

#     #append all padded embedding to a general list
#     all_padded_embeddings.append(padded_participant_embeddings)
#NO ERROR

In [ ]:
participant_groups = merged_data.groupby('Participant_ID')

# Initialize all_padded_embeddings as a list
all_padded_embeddings = []


# Iterate through each participant group and pad embeddings
for participant_id, group_data in participant_groups:
    participant_embeddings = group_data['embeddings'].tolist()
    participant_embeddings = np.array(participant_embeddings)

    # Use pad_sequences_uniform for padding and reshaping
    padded_participant_embeddings = pad_sequences_uniform(participant_embeddings)

     # Extend all_padded_embeddings with padded embeddings for this participant
    all_padded_embeddings.extend(padded_participant_embeddings)

"""Explanation of Changes:

Initialize as a list: We now initialize all_padded_embeddings as an empty list [] instead of a NumPy array.
Append to the list: Inside the loop, we use all_padded_embeddings.append(padded_participant_embeddings) to add the padded embeddings for each participant to the list.
Concatenate into an array: After the loop, we use np.concatenate(all_padded_embeddings, axis=0) to combine all the padded embeddings from the list into a single NumPy array.
"""
# Convert to list of lists for DataFrame assignment
all_padded_embeddings = [emb.tolist() for emb in all_padded_embeddings]

# Assign the list of lists to the DataFrame column
merged_data['padded_embeddings'] = all_padded_embeddings
#ValueError: cannot reshape array of size 76800 into shape (192,400,512)


In [ ]:
#Due to this error
"""ValueError                                Traceback (most recent call last)
<ipython-input-67-3dd57b6f7d51> in <cell line: 3>()
      2 X_train_reshaped = []
      3 for x in X_train:
----> 4   X_train_reshaped.append(np.array(x).reshape(MAX_TIMESTEPS, EMBEDDING_DIM))
      5 X_train = np.array(X_train_reshaped)
      6

ValueError: cannot reshape array of size 400 into shape (400,512)"""
#PROBELM WITH ABOVE METHOD OF APPEND
"""The Problem:

The core issue is that you are trying to directly assign a large NumPy array (all_padded_embeddings) as values to a column in a pandas DataFrame (merged_data['padded_embeddings']). This can be inefficient and may not always work as intended.

Why This Happens:

Pandas DataFrames are designed to store data in a tabular format, where each column typically contains values of a consistent data type. When you try to assign a large NumPy array to a single column, pandas might encounter difficulties in handling the complex data structure.

Suggested Solution:

Instead of directly assigning the NumPy array, consider converting it to a list of lists, where each inner list represents the padded embeddings for a particular row in the DataFrame. This way, pandas can manage the data more effectively. """

"""participant_groups = merged_data.groupby('Participant_ID')

all_padded_embeddings = []
# Iterate through each participant group
for participant_id, group_data in participant_groups:
    # Get all embeddings for the participant
    participant_embeddings = group_data['embeddings'].tolist()

    # Pad the embeddings for the participant
    # Before padding, make sure each element has the correct dimensionality
    padded_participant_embeddings = pad_sequences(
        [np.array(emb) for emb in participant_embeddings],  # Ensure each element is a numpy array
        maxlen=MAX_TIMESTEPS,
        dtype='float32',
        padding='post',
        truncating='post'
    )

    # Extend all_padded_embeddings with the padded embeddings for this participant
    all_padded_embeddings.extend(padded_participant_embeddings)

# Convert the list of padded embeddings to a list of lists
all_padded_embeddings = [emb.tolist() for emb in all_padded_embeddings]  # Convert to list of lists

# Now all_padded_embeddings should have the correct shape
merged_data['padded_embeddings'] = all_padded_embeddings  # Assign list of lists to DataFrame column"""

"participant_groups = merged_data.groupby('Participant_ID')\n\nall_padded_embeddings = []\n# Iterate through each participant group\nfor participant_id, group_data in participant_groups:\n    # Get all embeddings for the participant\n    participant_embeddings = group_data['embeddings'].tolist()\n\n    # Pad the embeddings for the participant\n    # Before padding, make sure each element has the correct dimensionality\n    padded_participant_embeddings = pad_sequences(\n        [np.array(emb) for emb in participant_embeddings],  # Ensure each element is a numpy array\n        maxlen=MAX_TIMESTEPS,\n        dtype='float32',\n        padding='post',\n        truncating='post'\n    )\n\n    # Extend all_padded_embeddings with the padded embeddings for this participant\n    all_padded_embeddings.extend(padded_participant_embeddings)\n\n# Convert the list of padded embeddings to a list of lists\nall_padded_embeddings = [emb.tolist() for emb in all_padded_embeddings]  # Convert to list of li

In [ ]:
# # Transform the list into a single array
# all_padded_embeddings = np.concatenate(all_padded_embeddings, axis=0)
# # Reshape to (samples, timesteps, embedding_dim)

# # Get the total number of samples
# num_samples = all_padded_embeddings.shape[0] // MAX_TIMESTEPS

# # Reshape considering the embedding dimension
# all_padded_embeddings = all_padded_embeddings.reshape(num_samples, MAX_TIMESTEPS, EMBEDDING_DIM)
#Error :ValueError: cannot reshape array of size 2854000 into shape (7135,400,512)


# merged_data['padded_embeddings'] = list(all_padded_embeddings)

In [ ]:
# Drop rows with missing PHQ8_Score if necessary (optional)
merged_data = merged_data.dropna(subset=['PHQ8_Score'])

# Fill missing values in numeric columns with the column's mean
numeric_columns = merged_data.select_dtypes(include=['float64', 'int64']).columns
merged_data[numeric_columns] = merged_data[numeric_columns].fillna(merged_data[numeric_columns].mean())
# Reset index
merged_data.reset_index(drop=True, inplace=True)  # Reset the index after dropping rows


In [ ]:
# Split the data
# Split the data
X = np.stack(merged_data['padded_embeddings'].values)  # Now we transform the values to an array of arrays
y = merged_data['PHQ8_Binary']
# Check shape of X before the split
print("Shape of X before the split:", X.shape)
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42
)
# Reshape X_train and X_val to ensure correct dimensionality
X_train = X_train.reshape(-1, MAX_TIMESTEPS, EMBEDDING_DIM)
X_val = X_val.reshape(-1, MAX_TIMESTEPS, EMBEDDING_DIM)

In [ ]:
# Check the type and shape of data
print(f"X_train type: {type(X_train)}, shape: {np.array(X_train).shape}")
print(f"X_val type: {type(X_val)}, shape: {np.array(X_val).shape}")
print(f"y_train type: {type(y_train)}, shape: {np.array(y_train).shape}")
print(f"y_val type: {type(y_val)}, shape: {np.array(y_val).shape}")


In [ ]:
# #why no need they are already
# # Ensure X_train, X_val are numpy arrays
# X_train = np.array(X_train)
# X_val = np.array(X_val)

In [ ]:
# Ensure y_train, y_val are numpy arrays and the correct shape
y_train = np.array(y_train).reshape(-1, 1)  # Reshape if needed for binary classification
y_val = np.array(y_val).reshape(-1, 1)  # Reshape if needed for binary classification

In [ ]:
print(f"X_train shape: {X_train.shape}")
print(f"X_val shape: {X_val.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_val shape: {y_val.shape}")

In [ ]:
# ## # Define input shape for the LSTM layer
# ## input_shape = (MAX_TIMESTEPS, EMBEDDING_DIM)  # (400, 512)

# ## # Define the input layer
# ## inputs = Input(shape=input_shape)

# # Define the input layer with the input shape
# inputs = Input(shape=(MAX_TIMESTEPS, EMBEDDING_DIM)) # input shape

# # Now, instead of input_shape in the first layer, you'll pass 'inputs' to avoid the only warning
# # Define the model layers, starting with the Bidirectional LSTM layer
# model = Sequential([
#     ### inputs, # Add the input layer
#     Bidirectional(LSTM(200, return_sequences=True))(inputs),  # Pass inputs to the LSTM layer
#     # Bidirectional(LSTM(200, return_sequences=True, input_shape=(MAX_TIMESTEPS, EMBEDDING_DIM))),
#     Bidirectional(LSTM(200)),
#     Dense(500, activation='relu'),
#     Dense(100, activation='relu'),
#     Dense(60, activation='relu'),
#     Dense(1, activation='sigmoid')  # For binary classification
# ])
# # Compile the model
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# """#Warning
# /usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
#   super().__init__(**kwargs)"""

# """#Warning2
# Okay, I understand that you are still encountering the same warning,even after implementing
#      the previous fix.I think I see the issue. Despite defining an Input layer, it's added as a
#      layer within the Sequential model instead of being passed to the first layer."""


In [ ]:
# Define input shape for the LSTM layer
input_shape = (MAX_TIMESTEPS, EMBEDDING_DIM)  # (400, 512)

# Define the input layer
inputs = Input(shape=input_shape)
# Now, instead of input_shape in the first layer, you'll pass 'inputs' to avoid the only warning
# Define the model layers, starting with the Bidirectional LSTM layer
model = Sequential()
model.add(Bidirectional(LSTM(200, return_sequences=True), input_shape=input_shape))
model.add(Bidirectional(LSTM(200)))
model.add(Dense(500, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(60, activation='relu'))
model.add(Dense(1, activation='sigmoid'))  # For binary classification


# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Reshape the padded embeddings within each row of X_train and X_val
X_train_reshaped = []
for x in X_train:
  X_train_reshaped.append(np.array(x).reshape(MAX_TIMESTEPS, EMBEDDING_DIM))
X_train = np.array(X_train_reshaped)

X_val_reshaped = []
for x in X_val:
  X_val_reshaped.append(np.array(x).reshape(MAX_TIMESTEPS, EMBEDDING_DIM))
X_val = np.array(X_val_reshaped)

# Convert your NumPy arrays to TensorFlow tensors
X_train_tensor = tf.convert_to_tensor(X_train, dtype=tf.float32)
y_train_tensor = tf.convert_to_tensor(y_train, dtype=tf.float32)
X_val_tensor = tf.convert_to_tensor(X_val, dtype=tf.float32)
y_val_tensor = tf.convert_to_tensor(y_val, dtype=tf.float32)

# Train the model
history = model.fit(X_train_tensor, y_train_tensor, validation_data=(X_val_tensor, y_val_tensor), epochs=20, batch_size=32)


In [ ]:
# Reshape X_train and X_val to include the embedding dimension
X_train = X_train.reshape(-1, MAX_TIMESTEPS, EMBEDDING_DIM)
X_val = X_val.reshape(-1, MAX_TIMESTEPS, EMBEDDING_DIM)

# Convert your NumPy arrays to TensorFlow tensors
X_train_tensor = tf.convert_to_tensor(X_train, dtype=tf.float32)
y_train_tensor = tf.convert_to_tensor(y_train, dtype=tf.float32)
X_val_tensor = tf.convert_to_tensor(X_val, dtype=tf.float32)
y_val_tensor = tf.convert_to_tensor(y_val, dtype=tf.float32)

# Train the model
history = model.fit(X_train_tensor, y_train_tensor, validation_data=(X_val_tensor, y_val_tensor), epochs=20, batch_size=32)




---



---



In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import logging
from typing import List, Dict, Tuple, Optional
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re
from tqdm import tqdm
import random
import tensorflow as tf
import tensorflow_hub as hub

def set_seed(seed: int = 42):
    np.random.seed(seed)
    torch.manual_seed(seed)
    random.seed(seed)
    tf.random.set_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
set_seed()

class TranscriptProcessor:
    def __init__(self, data_dir: str):
        self.data_dir = Path(data_dir)
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        # Load Universal Sentence Encoder
        self.encoder = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
        nltk.download('punkt')
        nltk.download('stopwords')
        self.stopwords = set(stopwords.words('english'))

    def load_transcripts(self, start_id: int = 300, end_id: int = 352, exclude_ids: List[int] = [342, 344]) -> pd.DataFrame:
        """Load and combine transcript files"""
        all_transcripts = []
        for session_id in range(start_id, end_id + 1):
            if session_id in exclude_ids:
                continue
            file_path = self.data_dir / f"{session_id}_TRANSCRIPT.csv"
            try:
                transcript = pd.read_csv(file_path, sep='\t')
                transcript['Participant_ID'] = session_id
                all_transcripts.append(transcript)
            except Exception as e:
                logging.warning(f"Error loading transcript {session_id}: {e}")
        return pd.concat(all_transcripts, ignore_index=True)

    def preprocess_text(self, text: str) -> str:
        """Clean and normalize text"""
        if isinstance(text, float):
            text = str(text)
        text = text.lower()
        text = re.sub(r'[^a-zA-Z\s]', '', text)
        tokens = word_tokenize(text)
        tokens = [t for t in tokens if t not in self.stopwords]
        return ' '.join(tokens)

    def generate_embeddings(self, texts: List[str]) -> np.ndarray:
        """Generate embeddings using Universal Sentence Encoder"""
        # Process in batches to handle memory constraints
        batch_size = 32
        embeddings = []

        for i in range(0, len(texts), batch_size):
            batch_texts = texts[i:i + batch_size]
            batch_embeddings = self.encoder(batch_texts)
            embeddings.append(batch_embeddings.numpy())

        return np.vstack(embeddings)

class DepressionDataset(Dataset):
    def __init__(self, embeddings: np.ndarray, labels: np.ndarray, device: torch.device):
        self.embeddings = torch.FloatTensor(embeddings).to(device)
        self.labels = torch.FloatTensor(labels).to(device)

    def __len__(self):
        return len(self.embeddings)

    def __getitem__(self, idx):
        return self.embeddings[idx], self.labels[idx]

class AttentionLSTM(nn.Module):
    """LSTM with attention mechanism for depression detection"""
    def __init__(self, input_size: int = 512, hidden_size: int = 200, num_layers: int = 2):
        super().__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers,
                           batch_first=True, bidirectional=True)
        self.attention = nn.MultiheadAttention(hidden_size * 2, num_heads=8)
        self.classifier = nn.Sequential(
            nn.Linear(hidden_size * 2, hidden_size),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(hidden_size, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        # Add sequence dimension for attention if needed
        if len(x.shape) == 2:
            x = x.unsqueeze(1)

        lstm_out, _ = self.lstm(x)
        attn_out, _ = self.attention(lstm_out, lstm_out, lstm_out)
        pooled = torch.mean(attn_out, dim=1)
        return self.classifier(pooled)

class DepressionDetectionPipeline:
    def __init__(self, data_dir: str):
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.processor = TranscriptProcessor(data_dir)

    def prepare_data(self) -> Tuple[np.ndarray, np.ndarray]:
        """Prepare data for model training"""
        transcripts = self.processor.load_transcripts()
        metadata = pd.read_csv(self.processor.data_dir / '/content/drive/MyDrive/datasset/transcript/train_split_Depression_AVEC2017 (1).csv')
        merged_data = transcripts.merge(metadata, on='Participant_ID')
        merged_data['cleaned_text'] = merged_data['value'].apply(self.processor.preprocess_text)
        embeddings = self.processor.generate_embeddings(merged_data['cleaned_text'].tolist())
        labels = merged_data['PHQ8_Binary'].values
        return embeddings, labels

    def train_model(self, embeddings: np.ndarray, labels: np.ndarray,
                   hidden_size: int = 200, batch_size: int = 32,
                   num_epochs: int = 20) -> Tuple[AttentionLSTM, Dict]:
        X_train, X_val, y_train, y_val = train_test_split(embeddings, labels, test_size=0.2)

        train_dataset = DepressionDataset(X_train, y_train, self.device)
        val_dataset = DepressionDataset(X_val, y_val, self.device)

        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=batch_size)

        model = AttentionLSTM(input_size=512, hidden_size=hidden_size).to(self.device)
        criterion = nn.BCELoss()
        optimizer = torch.optim.Adam(model.parameters())

        best_val_loss = float('inf')
        metrics_history = {'train_loss': [], 'val_loss': [], 'val_f1': []}

        for epoch in range(num_epochs):
            # Training loop implementation remains the same...
            model.train()
            train_loss = 0
            for batch_x, batch_y in train_loader:
                optimizer.zero_grad()
                outputs = model(batch_x)
                loss = criterion(outputs.squeeze(), batch_y)
                loss.backward()
                optimizer.step()
                train_loss += loss.item()

            # Validation loop implementation remains the same...
            model.eval()
            val_loss = 0
            val_preds = []
            val_true = []

            with torch.no_grad():
                for batch_x, batch_y in val_loader:
                    outputs = model(batch_x)
                    val_loss += criterion(outputs.squeeze(), batch_y).item()
                    val_preds.extend(outputs.squeeze().cpu().numpy())
                    val_true.extend(batch_y.cpu().numpy())

            val_f1 = f1_score(val_true, np.round(val_preds))

            metrics_history['train_loss'].append(train_loss / len(train_loader))
            metrics_history['val_loss'].append(val_loss / len(val_loader))
            metrics_history['val_f1'].append(val_f1)

            print(f"Epoch {epoch+1}/{num_epochs}")
            print(f"Train Loss: {train_loss/len(train_loader):.4f}")
            print(f"Val Loss: {val_loss/len(val_loader):.4f}")
            print(f"Val F1: {val_f1:.4f}")

            if val_loss < best_val_loss:
                best_val_loss = val_loss
                torch.save(model.state_dict(), 'best_model.pt')

        return model, metrics_history

def main():
    pipeline = DepressionDetectionPipeline('/content/drive/MyDrive/datasset/transcript')
    embeddings, labels = pipeline.prepare_data()
    model, metrics = pipeline.train_model(embeddings, labels)
    pd.DataFrame(metrics).to_csv('training_metrics.csv', index=False)

if __name__ == "__main__":
    main()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Epoch 1/20
Train Loss: 0.6591
Val Loss: 0.6398
Val F1: 0.0000
Epoch 2/20
Train Loss: 0.6575
Val Loss: 0.6412
Val F1: 0.0000
Epoch 3/20
Train Loss: 0.6557
Val Loss: 0.6436
Val F1: 0.0000
Epoch 4/20
Train Loss: 0.6563
Val Loss: 0.6392
Val F1: 0.0000
Epoch 5/20
Train Loss: 0.6563
Val Loss: 0.6402
Val F1: 0.0000
Epoch 6/20
Train Loss: 0.6572
Val Loss: 0.6449
Val F1: 0.0000
Epoch 7/20
Train Loss: 0.6576
Val Loss: 0.6423
Val F1: 0.0000
Epoch 8/20
Train Loss: 0.6552
Val Loss: 0.6391
Val F1: 0.0000
Epoch 9/20
Train Loss: 0.6551
Val Loss: 0.6481
Val F1: 0.0000
Epoch 10/20
Train Loss: 0.6565
Val Loss: 0.6395
Val F1: 0.0000
Epoch 11/20
Train Loss: 0.6556
Val Loss: 0.6414
Val F1: 0.0000
Epoch 12/20
Train Loss: 0.6554
Val Loss: 0.6398
Val F1: 0.0000
Epoch 13/20
Train Loss: 0.6555
Val Loss: 0.6419
Val F1: 0.0000
Epoch 14/20
Train Loss: 0.6552
Val Loss: 0.6405
Val F1: 0.0000
Epoch 15/20
Train Loss: 0.6572
Val Loss: 0.6424
Val F1: 0.0000
Epoch 16/20
Train Loss: 0.6568
Val Loss: 0.6399
Val F1: 0.0000
E